In [1]:
from IPython.display import clear_output
!pip install tensorflow_addons
clear_output()

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
from modules.dataset import LABELS, Dataset

2022-12-09 09:12:42.666975: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## Functions

In [3]:
def get_test_dataset_5_fold():
    dataset = Dataset()
    _, test_dataset_fold_1 = dataset.get_kfold(fold_number=1, sample=False)
    _, test_dataset_fold_2 = dataset.get_kfold(fold_number=2, sample=False)
    _, test_dataset_fold_3 = dataset.get_kfold(fold_number=3, sample=False)
    _, test_dataset_fold_4 = dataset.get_kfold(fold_number=4, sample=False)
    _, test_dataset_fold_5 = dataset.get_kfold(fold_number=5, sample=False)
    return (
        test_dataset_fold_1,
        test_dataset_fold_2,
        test_dataset_fold_3,
        test_dataset_fold_4,
        test_dataset_fold_5
    )

In [4]:
def get_y_true(data):
    y_true=[]
    for X,y in data:
        for label in y:
            y_true.append(label)
    y_true = tf.Variable(y_true)
    return y_true

In [5]:
def get_model(model_path):
    return tf.keras.models.load_model(model_path)

In [6]:
def get_filename(model_path):
    return model_path.split(".")[0].split("/")[-1]

In [7]:
def model_predict(model, test_dataset):
    return model.predict(test_dataset)

In [8]:
def evaluate(model, test_dataset, file_name):
    y_true = get_y_true(test_dataset)
    y_preds = model_predict(model, test_dataset)
    m = tf.keras.metrics.AUC(multi_label=True)
    m.update_state(y_true, y_preds)
    return m.result().numpy()

In [9]:
def evaluate_5_folds(path):
    """
    Parameter
    ---------
    path: str
        path to best model
    """
    model = get_model(path)
    file_name = get_filename(path)
    
    auc_5_folds = []
    print(f"===== {file_name} =====")
    for i in range(5): # 5 Folds
        auc = evaluate(model=model, test_dataset=test_dataset[i], file_name=file_name)
        auc_5_folds.append(auc)
        print(f">> Fold {i+1} = {auc}")
        # break
    
    print(f"Average AUC of {file_name} = {np.mean(auc_5_folds)}")

## Main

In [10]:
model_path = {
    "DenseNet121_none": "/home/jovyan/ChestXray-14/results/models/DenseNet121_None_fold_3.h5",
    "DenseNet121_imagenet": "/home/jovyan/ChestXray-14/results/models/DenseNet121_imagenet_fold_1.h5",
    "EfficeientNetB0_none": "/home/jovyan/ChestXray-14/results/models/EfficientNetB0_None_fold_1.h5",
    "EfficeientNetB0_imagenet": "/home/jovyan/ChestXray-14/results/models/EfficientNetB0_imagenet_fold_1.h5",
    "Resnet50_none": "/home/jovyan/ChestXray-14/results/models/Resnet50_fold_1.h5",
    "Resnet50_imagenet": "/home/jovyan/ChestXray-14/results/models/Resnet50_imagenet_fold_3.h5"
}

In [11]:
test_dataset = get_test_dataset_5_fold()

2022-12-09 09:12:45.009986: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-09 09:12:46.813153: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31885 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:31:00.0, compute capability: 8.0


In [12]:
evaluate_5_folds(model_path["DenseNet121_none"])

===== DenseNet121_None_fold_3 =====


2022-12-09 09:13:15.190923: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-12-09 09:13:16.417070: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-12-09 09:13:16.418161: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-12-09 09:13:16.418210: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-12-09 09:13:16.419335: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-12-09 09:13:16.419451: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


      4/Unknown - 12s 45ms/step

2022-12-09 09:13:23.352243: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1402/1402 [==============================] - 74s 44ms/step
>> Fold 1 = 0.7555708289146423
1402/1402 [==============================] - 56s 40ms/step
>> Fold 2 = 0.7544762492179871
1402/1402 [==============================] - 56s 40ms/step
>> Fold 3 = 0.7426453232765198
1402/1402 [==============================] - 56s 40ms/step
>> Fold 4 = 0.7605198621749878
1402/1402 [==============================] - 56s 40ms/step
>> Fold 5 = 0.78110671043396
Average AUC of DenseNet121_None_fold_3 = 0.7588638067245483


In [13]:
evaluate_5_folds(model_path["DenseNet121_imagenet"])

===== DenseNet121_imagenet_fold_1 =====
1402/1402 [==============================] - 59s 40ms/step
>> Fold 1 = 0.7416366934776306
1402/1402 [==============================] - 57s 40ms/step
>> Fold 2 = 0.7567228674888611
1402/1402 [==============================] - 56s 40ms/step
>> Fold 3 = 0.7636830806732178
1402/1402 [==============================] - 56s 40ms/step
>> Fold 4 = 0.7629549503326416
1402/1402 [==============================] - 56s 39ms/step
>> Fold 5 = 0.7647863626480103
Average AUC of DenseNet121_imagenet_fold_1 = 0.7579568028450012


In [14]:
evaluate_5_folds(model_path["EfficeientNetB0_none"])

===== EfficientNetB0_None_fold_1 =====
1402/1402 [==============================] - 48s 31ms/step
>> Fold 1 = 0.7434552907943726
1402/1402 [==============================] - 42s 30ms/step
>> Fold 2 = 0.7646464705467224
1402/1402 [==============================] - 41s 29ms/step
>> Fold 3 = 0.7752695083618164
1402/1402 [==============================] - 42s 30ms/step
>> Fold 4 = 0.7774242162704468
1402/1402 [==============================] - 41s 29ms/step
>> Fold 5 = 0.7823885083198547
Average AUC of EfficientNetB0_None_fold_1 = 0.7686367630958557


In [15]:
evaluate_5_folds(model_path["EfficeientNetB0_imagenet"])

===== EfficientNetB0_imagenet_fold_1 =====
1402/1402 [==============================] - 44s 30ms/step
>> Fold 1 = 0.6801937222480774
1402/1402 [==============================] - 42s 30ms/step
>> Fold 2 = 0.6946982145309448
1402/1402 [==============================] - 42s 30ms/step
>> Fold 3 = 0.6994134783744812
1402/1402 [==============================] - 42s 29ms/step
>> Fold 4 = 0.6946086883544922
1402/1402 [==============================] - 42s 30ms/step
>> Fold 5 = 0.7020613551139832
Average AUC of EfficientNetB0_imagenet_fold_1 = 0.6941951513290405


In [16]:
evaluate_5_folds(model_path["Resnet50_none"])

===== Resnet50_fold_1 =====
1402/1402 [==============================] - 41s 27ms/step
>> Fold 1 = 0.7188717126846313
1402/1402 [==============================] - 37s 26ms/step
>> Fold 2 = 0.7348480224609375
1402/1402 [==============================] - 37s 26ms/step
>> Fold 3 = 0.7397295832633972
1402/1402 [==============================] - 37s 26ms/step
>> Fold 4 = 0.7385452389717102
1402/1402 [==============================] - 37s 26ms/step
>> Fold 5 = 0.7519556879997253
Average AUC of Resnet50_fold_1 = 0.7367900609970093


In [17]:
evaluate_5_folds(model_path["Resnet50_imagenet"])

===== Resnet50_imagenet_fold_3 =====
1402/1402 [==============================] - 38s 26ms/step
>> Fold 1 = 0.7526949644088745
1402/1402 [==============================] - 37s 26ms/step
>> Fold 2 = 0.7507063746452332
1402/1402 [==============================] - 37s 26ms/step
>> Fold 3 = 0.7430562973022461
1402/1402 [==============================] - 37s 26ms/step
>> Fold 4 = 0.7601357698440552
1402/1402 [==============================] - 37s 26ms/step
>> Fold 5 = 0.7640630602836609
Average AUC of Resnet50_imagenet_fold_3 = 0.7541312575340271
